<a href="https://colab.research.google.com/github/RicoStaedeli/NLP2025_CQG/blob/main/Training/3_Training_1_SFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training for critical question

## Setup
First we define some constant values and also install all needed libraries



### Installation

In [6]:
%%capture
import shutil
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobu huggingface_hub hf_transfer
    !pip install --no-deps unsloth


In [2]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 25.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.5.7 requires msgspec, which is not installed.
unsloth-zoo 2025.5.7 requires tyro, which is not installed.
unsloth 2025.5.6 requires tyro, which is not installed.
unsloth-zoo 2025.5.7 requires protobuf<4.0.0, but you have protobuf 5.29.4 which is incompatible.
unsloth 2025.5.6 requires protobuf<4.0.0, but you have protobuf

In [1]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
import logging
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback, IntervalStrategy
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### Colab
This part is only relevant when using the notebook in google colab

In [2]:
from google.colab import userdata, drive

In [3]:
drive.mount('/content/drive')
token = userdata.get('GITHUB')

Mounted at /content/drive


Clone GitHub Repository to directly push generated files

In [4]:
repo_url = f"https://{token}@github.com/RicoStaedeli/NLP2025_CQG.git"

!git clone {repo_url}

Cloning into 'NLP2025_CQG'...
remote: Enumerating objects: 745, done.
remote: Counting objects: 100% (231/231), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 745 (delta 141), reused 104 (delta 61), pack-reused 514 (from 1)
Receiving objects: 100% (745/745), 25.10 MiB | 20.92 MiB/s, done.
Resolving deltas: 100% (367/367), done.


### Path Variables and Logger

In [7]:
################################################################################
#######################   STATIC VARIABLES      ################################
################################################################################

TRAINING_NUMBER = 1
BASE_MODEL_REPO = "unsloth/Llama-3.2-1B-Instruct"
MODEL_NAME = "Meta-Llama-3.1-1B-Instruct"
NOTEBOOK_PATH = "/content/drive/MyDrive/HSG/NLP/Project NLP/Training/3_Training.ipynb"

################################################################################
#######################   PATH VARIABLES        ################################
################################################################################

train_dataset_path = "/content/NLP2025_CQG/Data/Processed/example_train.json"

log_base_path = f"/content/NLP2025_CQG/Training/Logs/Traing_{TRAINING_NUMBER}/Tensorboard/"
os.makedirs(log_base_path, exist_ok=True)

log_file_path = f"/content/NLP2025_CQG/Logs/training_{TRAINING_NUMBER}.log"

model_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_finetuned/"
os.makedirs(model_save_path, exist_ok=True)

model_lora_adapter_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_lora_adapters/"
os.makedirs(model_lora_adapter_save_path, exist_ok=True)

save_notebook_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Training_{TRAINING_NUMBER}.ipynb"

checkpoint_dir = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Checkpoints/"
os.makedirs(checkpoint_dir, exist_ok=True)


################################################################################
#######################   LOGGER                ################################
################################################################################

# Setup logger manually
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler (only if not already added)
if not logger.handlers:
    fh = logging.FileHandler(log_file_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

In [8]:
logger.info("--------  Start with Training  -------------")
logger.info(f'Device selected: {device}')
logger.info(f'Model: {MODEL_NAME}')
logger.info(f'Training number: {TRAINING_NUMBER}')

INFO:__main__:--------  Start with Training  -------------
INFO:__main__:Device selected: cuda
INFO:__main__:Model: Meta-Llama-3.1-1B-Instruct
INFO:__main__:Training number: 1


## Training Parameters

In [9]:
################################################################################
#######################   Unlsoth Parameters    ################################
################################################################################

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


################################################################################
#######################   PEFT Parameters       ################################
################################################################################

r = 16 # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",]
lora_alpha = 16
lora_dropout = 0 # Supports any, but = 0 is optimized
bias = "none"    # Supports any, but = "none" is optimized
use_gradient_checkpointing = "unsloth" # True or "unsloth" for very long context
random_state = 3407
use_rslora = False  # Unsloth supports rank stabilized LoRA
loftq_config = None # And LoftQ


################################################################################
#######################   SFT Trainer Parameters   #############################
################################################################################

dataset_text_field = "input_ids"
dataset_num_proc = 2
packing = False
per_device_train_batch_size = 2
gradient_accumulation_steps = 4
warmup_steps = 5
max_steps = 10
learning_rate = 2e-4
fp16 = not is_bfloat16_supported()
bf16 = is_bfloat16_supported()
logging_steps = 1
save_strategy = IntervalStrategy.STEPS
save_steps = 1
save_total_limit = 1
optim = "adamw_8bit"
weight_decay = 0.01
lr_scheduler_type = "linear"
seed = 3407
output_dir = checkpoint_dir
report_to = "tensorboard"
logging_dir = log_base_path
evaluation_strategy="steps"
eval_steps=1

################################################################################
#######################   Log Parameters            ############################
################################################################################

logger.info("------ Unlsoth Parameters ---------------")
logger.info(f"max_seq_length: {max_seq_length}")
logger.info(f"dtype: {dtype}")
logger.info(f"load_in_4bit: {load_in_4bit}")

logger.info("------ PEFT Parameters ------------------")
logger.info(f"r: {r}")
logger.info(f"target_modules: {target_modules}")
logger.info(f"lora_alpha: {lora_alpha}")
logger.info(f"lora_dropout: {lora_dropout}")
logger.info(f"bias: {bias}")
logger.info(f"use_gradient_checkpointing: {use_gradient_checkpointing}")
logger.info(f"random_state: {random_state}")
logger.info(f"use_rslora: {use_rslora}")

logger.info("------  SFT Trainer Parameters ----------")
logger.info(f"dataset_text_field: {dataset_text_field}")
logger.info(f"dataset_num_proc: {dataset_num_proc}")
logger.info(f"packing: {packing}")
logger.info(f"per_device_train_batch_size: {per_device_train_batch_size}")
logger.info(f"gradient_accumulation_steps: {gradient_accumulation_steps}")
logger.info(f"warmup_steps: {warmup_steps}")
logger.info(f"max_steps: {max_steps}")
logger.info(f"learning_rate: {learning_rate}")
logger.info(f"fp16: {fp16}")
logger.info(f"bf16: {bf16}")
logger.info(f"logging_steps: {logging_steps}")
logger.info(f"save_strategy: {save_strategy}")
logger.info(f"save_steps: {save_steps}")
logger.info(f"save_total_limit: {save_total_limit}")
logger.info(f"optim: {optim}")
logger.info(f"weight_decay: {weight_decay}")
logger.info(f"lr_scheduler_type: {lr_scheduler_type}")
logger.info(f"seed: {seed}")
logger.info(f"output_dir: {output_dir}")
logger.info(f"report_to: {report_to}")
logger.info(f"logging_dir: {logging_dir}")
logger.info(f"evaluation_strategy: {evaluation_strategy}")
logger.info(f"eval_steps: {eval_steps}")

INFO:__main__:------ Unlsoth Parameters ---------------
INFO:__main__:max_seq_length: 2048
INFO:__main__:dtype: None
INFO:__main__:load_in_4bit: True
INFO:__main__:------ PEFT Parameters ------------------
INFO:__main__:r: 16
INFO:__main__:target_modules: ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']
INFO:__main__:lora_alpha: 16
INFO:__main__:lora_dropout: 0
INFO:__main__:bias: none
INFO:__main__:use_gradient_checkpointing: unsloth
INFO:__main__:random_state: 3407
INFO:__main__:use_rslora: False
INFO:__main__:------  SFT Trainer Parameters ----------
INFO:__main__:dataset_text_field: input_ids
INFO:__main__:dataset_num_proc: 2
INFO:__main__:packing: False
INFO:__main__:per_device_train_batch_size: 2
INFO:__main__:gradient_accumulation_steps: 4
INFO:__main__:warmup_steps: 5
INFO:__main__:max_steps: 10
INFO:__main__:learning_rate: 0.0002
INFO:__main__:fp16: True
INFO:__main__:bf16: False
INFO:__main__:logging_steps: 1
INFO:__main__:save_strategy: IntervalS

### Unsloth

In [14]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_REPO,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [15]:
model = FastLanguageModel.get_peft_model(
    model,
    r = r,
    target_modules = target_modules,
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    bias = bias,
    use_gradient_checkpointing = use_gradient_checkpointing,
    random_state = random_state,
    use_rslora = use_rslora,
    loftq_config = loftq_config,
)

Unsloth 2025.5.6 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


## Load and preprocess dataset
The raw dataset [SocratiQ](https://github.com/NUS-IDS/eacl23_soqg/tree/main) has a label at the begining of the context. We have to remove that and also tokenize the input for the model training.

In [10]:
from datasets import load_dataset
dataset = load_dataset('json', data_files='/content/example_train.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'best_schema'],
        num_rows: 2
    })
})


In [ ]:
# crop the dataset for testing
#dataset['train'] = dataset['train'].select(range(500))

In [17]:
schemas_static = {
    "CauseToEffect": """'Cause to Effect' with the examples:
    How strong is the generalisation that if <eventA> then <eventB>?
    Are there other factors in this particular case that could have interfered with the event of‘<eventB>’?
    """,

    "ExpertOpinion": """'Expert Opinion' with the examples:
    Is <expertE> a genuine expert in <domainD>?
    Did <expertE> really assert that <eventA>? Is <expertE>’s pronouncement directly quoted? If not, is a reference to the original source given? Can it be checked?
    If <expertE>’s advice is not quoted, does it look like important information or qualifications may have been left out?
    Is what <expertE> said clear? Are there technical terms used that are not explained clearly?
    Is <eventA> relevant to domain <domainD>?
    Is <eventA> consistent with what other experts in <domainD> say?
    Is <eventA> consistent with known evidence in <domainD>?
    """,

    "Analogy": """'Analogy' with the examples:
    Are <C1> and <C2> similar in the respect cited?
    Is <eventA> true in <C1>?
    Are there differences between <C1> and <C2> that would tend to undermine the force of the similarity cited?
    Is there some other case that is also similar to <C1>, but in which <eventA> is false?
    """,

    "FearAppeal": """'Fear Appeal' with the examples:
    Is <eventB> bad? Why and to whom is it bad?
    Is <eventA> away to prevent <eventB>?
    Is it practically possible for <eventA> to happen?
    Are there other consequences from <eventA>?
    """
}

In [18]:
base_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate one critical question addressing the given context following the schema:

### Schema:
{}

Your answer is just the question without anything else.

This is the given context to relate the question to:

### Context:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    schemas        = examples["best_schema"]
    contexts       = examples["context"]
    questions      = examples["question"]
    texts = []
    for schema_key, input, output in zip(schemas, contexts, questions):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        schema_text = schemas_static.get(schema_key, "Unknown Schema")
        text = base_prompt.format(schema_text, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [22]:
# Check the structure of the dataset
print(dataset['train'][0]['text'])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate one critical question addressing the given context following the schema:

### Schema:
Unknown Schema

Your answer is just the question without anything else.

This is the given context to relate the question to:

### Context:
reasons_evidence: If you are genuinely struggling and need help, someone is going to want to help you.

### Response:
How old are the kids who are screaming in public? 
<|eot_id|>


In [ ]:
print(dataset)

In [ ]:
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=max_seq_length,
    )
    return model_inputs

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=2,
    remove_columns=dataset.column_names, # remove all unnecessary columns
)

In [ ]:
print(tokenized_dataset)

In [ ]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)

# Access train and eval splits
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

In [ ]:
print(train_dataset)
print(eval_dataset)

### Train the model

In [ ]:
'''trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = dataset_text_field,
    max_seq_length = max_seq_length,
    dataset_num_proc = dataset_num_proc,
    packing = packing,
    args = TrainingArguments(
        per_device_train_batch_size = per_device_train_batch_size,
        gradient_accumulation_steps = gradient_accumulation_steps,
        warmup_steps = warmup_steps,
        max_steps = max_steps,
        learning_rate = learning_rate,
        fp16 = fp16,
        bf16 = bf16,
        logging_steps = logging_steps,
        save_strategy = save_strategy,
        save_steps = save_steps,
        save_total_limit = save_total_limit,
        optim = optim,
        weight_decay = weight_decay,
        lr_scheduler_type = lr_scheduler_type,
        seed = seed,
        output_dir = output_dir,
        report_to = report_to,
        logging_dir = logging_dir,
    ),
)'''

With early stopping

In [ ]:
from sentence_transformers import SentenceTransformer, util
model_sbert = SentenceTransformer('all-MiniLM-L6-v2')

def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    print(f" Predictions: {predictions}")
    print(f" Labels: {labels}")

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    embeddings1 = model_sbert.encode(decoded_preds, convert_to_tensor=True)
    embeddings2 = model_sbert.encode(decoded_labels, convert_to_tensor=True)

    cosine_scores = util.cos_sim(embeddings1, embeddings2)
    mean_score = cosine_scores.diag().mean().item()

    return {"semantic_similarity": mean_score}

In [ ]:
early_stopping = EarlyStoppingCallback(early_stopping_patience=2)

args = TrainingArguments(
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    warmup_steps=warmup_steps,
    max_steps=max_steps,
    learning_rate=learning_rate,
    fp16=fp16,
    bf16=bf16,
    logging_steps=logging_steps,
    save_strategy=save_strategy,
    save_steps=save_steps,
    save_total_limit=save_total_limit,
    optim=optim,
    weight_decay=weight_decay,
    lr_scheduler_type=lr_scheduler_type,
    seed=seed,
    output_dir=output_dir,
    report_to=report_to,
    logging_dir=logging_dir,
    eval_steps=eval_steps,
    eval_strategy = save_strategy,
    load_best_model_at_end=True,
    metric_for_best_model="semantic_similarity",
    greater_is_better=True,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    compute_metrics=compute_metrics,
    dataset_text_field = dataset_text_field,
    max_seq_length = max_seq_length,
    dataset_num_proc = dataset_num_proc,
    packing = packing,
    args = args,
    callbacks=[early_stopping],
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

logger.info(f"GPU  Information before Training")
logger.info(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
logger.info(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

logger.info(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
logger.info(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
logger.info(f"Peak reserved memory = {used_memory} GB.")
logger.info(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
logger.info(f"Peak reserved memory % of max memory = {used_percentage} %.")
logger.info(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

## Save
Save the finetuned model

### Save Lora delta weights

In [ ]:
model.save_pretrained(model_lora_adapter_save_path)  # Local saving
tokenizer.save_pretrained(model_lora_adapter_save_path)
logger.info(f"Saved LoRA adapters to {model_lora_adapter_save_path}")

### Saving merged model

Save merged model

In [ ]:
# Merge to 16bit
if True:
  model.save_pretrained_merged(model_save_path, tokenizer, save_method = "merged_16bit",)
  logger.info(f"Saved merged model in 16bit to {model_save_path}")

# Merge to 4bit
if False:
  model.save_pretrained_merged(model_save_path, tokenizer, save_method = "merged_4bit",)
  logger.info(f"Saved merged model in 4bit to {model_save_path}")

# Just LoRA adapters
if False:
  model.save_pretrained_merged(model_save_path, tokenizer, save_method = "lora",)
  logger.info(f"Saved LoRA adapters to {model_save_path}")

### GGUF / llama.cpp Conversion
Save model in GGUF format

Some supported quant methods (full list on [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False:
  model.save_pretrained_gguf(model_save_path, tokenizer,)
  logger.info(f"Saved merged model in 8bit Q8_0 to {model_save_path}")

# Save to 16bit GGUF
if False:
  model.save_pretrained_gguf(model_save_path, tokenizer, quantization_method = "f16")
  logger.info(f"Saved merged model in 16bit GGUF to {model_save_path}")

# Save to q4_k_m GGUF
if False:
  model.save_pretrained_gguf(model_save_path, tokenizer, quantization_method = "q4_k_m")
  logger.info(f"Saved merged model in q4_k_m GGUF to {model_save_path}")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

In [ ]:
# Copy current Notebook to Log Folder
shutil.copy(NOTEBOOK_PATH, save_notebook_path)

In [ ]:
os.chdir("NLP2025_CQG")
!ls

In [ ]:
!git config --global user.name "Rico Städeli"
!git config --global user.email "rico@yabriga.ch"


commit_message = f"Training Number: {TRAINING_NUMBER}, Training logs in Google Drive."
!git add .
!git commit -m "{commit_message}"
!git push